## Lab 2
### Part 2: Dealing with overfitting

Today we work with [Fashion-MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) (*hint: it is available in `torchvision`*).

Your goal for today:
1. Train a FC (fully-connected) network that achieves >= 0.885 test accuracy.
2. Cause considerable overfitting by modifying the network (e.g. increasing the number of network parameters and/or layers) and demonstrate in in the appropriate way (e.g. plot loss and accurasy on train and validation set w.r.t. network complexity).
3. Try to deal with overfitting (at least partially) by using regularization techniques (Dropout/Batchnorm/...) and demonstrate the results.

__Please, write a small report describing your ideas, tries and achieved results in the end of this file.__

*Note*: Tasks 2 and 3 are interrelated, in task 3 your goal is to make the network from task 2 less prone to overfitting. Task 1 is independent from 2 and 3.

*Note 2*: We recomment to use Google Colab or other machine with GPU acceleration.

In [42]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchsummary
from IPython.display import clear_output
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import os


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [43]:
# Technical function
def mkdir(path):
    if not os.path.exists(root_path):
        os.mkdir(root_path)
        print('Directory', path, 'is created!')
    else:
        print('Directory', path, 'already exists!')

root_path = 'fmnist'
mkdir(root_path)

Directory fmnist already exists!


In [44]:
download = True
train_transform = transforms.ToTensor()
test_transform = transforms.ToTensor()
transforms.Compose((transforms.ToTensor()))


fmnist_dataset_train = torchvision.datasets.FashionMNIST(root_path,
                                                        train=True,
                                                        transform=train_transform,
                                                        target_transform=None,
                                                        download=download)
fmnist_dataset_test = torchvision.datasets.FashionMNIST(root_path,
                                                       train=False,
                                                       transform=test_transform,
                                                       target_transform=None,
                                                       download=download)

In [45]:
train_loader = torch.utils.data.DataLoader(fmnist_dataset_train,
                                           batch_size=128,
                                           shuffle=True,
                                           num_workers=2)
test_loader = torch.utils.data.DataLoader(fmnist_dataset_test,
                                          batch_size=256,
                                          shuffle=False,
                                          num_workers=2)

In [46]:
len(fmnist_dataset_test)

10000

In [47]:
for img, label in train_loader:
    print(img.shape)
#     print(img)
    print(label.shape)
    print(label.size(0))
    break

torch.Size([128, 1, 28, 28])
torch.Size([128])
128


### Task 1
Train a network that achieves $\geq 0.885$ test accuracy. It's fine to use only Linear (`nn.Linear`) layers and activations/dropout/batchnorm. Convolutional layers might be a great use, but we will meet them a bit later.

In [48]:
class TinyNeuralNetwork(nn.Module):
    def __init__(self, input_shape=28*28, num_classes=10, input_channels=1):
        super(self.__class__, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(), # This layer converts image into a vector to use Linear layers afterwards
            # Your network structure comes here
            nn.Linear(input_shape, 24*24),
            nn.ReLU(),
            nn.Linear(24*24, 20*20),
            nn.ReLU(),
            nn.Linear(20*20, 15*15),
            nn.ReLU(),
            nn.Linear(15*15, num_classes),
        )

    def forward(self, inp):
        out = self.model(inp)

In [49]:
torchsummary.summary(TinyNeuralNetwork().to(device), (28*28,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 576]         452,160
              ReLU-3                  [-1, 576]               0
            Linear-4                  [-1, 400]         230,800
              ReLU-5                  [-1, 400]               0
            Linear-6                  [-1, 225]          90,225
              ReLU-7                  [-1, 225]               0
            Linear-8                   [-1, 10]           2,260
Total params: 775,445
Trainable params: 775,445
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.96
Estimated Total Size (MB): 2.99
----------------------------------------------------------------


Your experiments come here:

In [67]:
# Your experiments, training and validation loops here
def try_model(num_epochs, model, opt, loss_func):
    train_accuracies = []
    test_accuracies = []
    for epoch in range(num_epochs + 1):
        model.train()

        train_predicted = 0
        test_predicted = 0

        for images, labels in train_loader:
            outputs = model(images.to(device))
            labels = labels.to(device)

            loss = loss_func(outputs, labels)
            loss.backward()
            opt.step()
            opt.zero_grad()

            train_predicted += (torch.max(outputs, 1)[1] == labels).sum().item()

        model.eval()

        for images, labels in test_loader:
            outputs = model(images.to(device))
            labels = labels.to(device)

            test_predicted += (torch.max(outputs, 1)[1] == labels).sum().item()

        train_accuracy = train_predicted / len(fmnist_dataset_train)
        test_accuracy = test_predicted / len(fmnist_dataset_test)

        train_accuracies.append(train_accuracy)
        test_accuracies.append(test_accuracy)

        if epoch % 10 == 0:
            print(f"{epoch:03d}/{num_epochs}", f"Train accuracy: {train_accuracy:.5f} Test accuracy: {test_accuracy:.5f}")

    return train_accuracies, test_accuracies



In [68]:
model = TinyNeuralNetwork().to(device)
opt = torch.optim.SGD(model.parameters(), lr=1e-2) # YOUR CODE HERE
loss_func = nn.CrossEntropyLoss() # YOUR CODE HERE
num_epochs = 100

tiny_train_accuracies, tiny_test_accuracies = try_model(num_epochs, model, opt, loss_func)

TypeError: ignored

### Task 2: Overfit it.
Build a network that will overfit to this dataset. Demonstrate the overfitting in the appropriate way (e.g. plot loss and accurasy on train and test set w.r.t. network complexity).

*Note:* you also might decrease the size of `train` dataset to enforce the overfitting and speed up the computations.

In [69]:
class OverfittingNeuralNetwork(nn.Module):
    def __init__(self, input_shape=28*28, num_classes=10, input_channels=1):
        super(self.__class__, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(), # This layer converts image into a vector to use Linear layers afterwards
            # Your network structure comes here
           nn.Linear(input_shape, 25*25),
            nn.ReLU(),
            nn.Linear(25*25, 22*22),
            nn.ReLU(),
            nn.Linear(22*22, 19*19),
            nn.ReLU(),
            nn.Linear(19*19, 16*16),
            nn.ReLU(),
            nn.Linear(16*16, num_classes),
        )

    def forward(self, inp):
        out = self.model(inp)

In [70]:
torchsummary.summary(OverfittingNeuralNetwork().to(device), (28*28,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 625]         490,625
              ReLU-3                  [-1, 625]               0
            Linear-4                  [-1, 484]         302,984
              ReLU-5                  [-1, 484]               0
            Linear-6                  [-1, 361]         175,085
              ReLU-7                  [-1, 361]               0
            Linear-8                  [-1, 256]          92,672
              ReLU-9                  [-1, 256]               0
           Linear-10                   [-1, 10]           2,570
Total params: 1,063,936
Trainable params: 1,063,936
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 4.06
Estimat

In [71]:
model = OverfittingNeuralNetwork().to(device)
opt = torch.optim.SGD(model.parameters(), lr=1e-2) # YOUR CODE HERE
loss_func = nn.CrossEntropyLoss() # YOUR CODE HERE
num_epochs = 200

# Your experiments, come here
overfitted_train_accuracies, overfitted_test_accuracies = try_model(num_epochs, model, opt, loss_func)


TypeError: ignored

### Task 3: Fix it.
Fix the overfitted network from the previous step (at least partially) by using regularization techniques (Dropout/Batchnorm/...) and demonstrate the results.

In [72]:
class FixedNeuralNetwork(nn.Module):
    def __init__(self, input_shape=28*28, num_classes=10, input_channels=1):
        super(self.__class__, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(), # This layer converts image into a vector to use Linear layers afterwards
            # Your network structure comes here
             nn.Linear(input_shape, 25*25),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(25*25, 22*22),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(22*22, 19*19),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(19*19, 16*16),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(16*16, num_classes),
        )

    def forward(self, inp):
        out = self.model(inp)

In [73]:
torchsummary.summary(FixedNeuralNetwork().to(device), (28*28,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 625]         490,625
              ReLU-3                  [-1, 625]               0
           Dropout-4                  [-1, 625]               0
            Linear-5                  [-1, 484]         302,984
              ReLU-6                  [-1, 484]               0
           Dropout-7                  [-1, 484]               0
            Linear-8                  [-1, 361]         175,085
              ReLU-9                  [-1, 361]               0
          Dropout-10                  [-1, 361]               0
           Linear-11                  [-1, 256]          92,672
             ReLU-12                  [-1, 256]               0
          Dropout-13                  [-1, 256]               0
           Linear-14                   

In [74]:
model = FixedNeuralNetwork().to(device)
opt = torch.optim.SGD(model.parameters(), lr=1e-2) # YOUR CODE HERE
loss_func = nn.CrossEntropyLoss() # YOUR CODE HERE
num_epochs = 100
# Your experiments, come here
fixed_train_accuracies, fixed_test_accuracies = try_model(num_epochs, model, opt, loss_func)


TypeError: ignored

### Conclusions:
_Write down small report with your conclusions and your ideas._

4 линейный слои и активации, проходя каждый слой мы получим тест примерно _____

если увиличить слои до 5 то модель переобучивается. трейн растет, тест не меняется . Видим что сеть подстраивается под датасет.

Слои Дропоут мешают переобучиваться сети .